In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np 
import matplotlib.pyplot as plt 
import math
from sklearn.model_selection import train_test_split 
from sklearn.impute import SimpleImputer
import warnings 
warnings.filterwarnings("ignore")
import acquire
import seaborn as sns 
import pydataset as p
import os

### Classification

Preparation

In [16]:
iris_df = acquire.get_iris_data()

Acquiring data from SQL database


In [17]:
iris_df.head(2)

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa


In [18]:
iris_df.columns

Index(['species_id', 'measurement_id', 'sepal_length', 'sepal_width',
       'petal_length', 'petal_width', 'species_name'],
      dtype='object')

In [19]:
#Drop the species_id and measurement_id columns 
#dataframe.drop_paranthesis_columnsequals_brackets_'columnName'_comma_implace=true_closed parenthesis
iris_df.drop(columns=['species_id'], inplace=True)
iris_df.head()

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [20]:
#drop the measurement_id column 
iris_df.drop(columns=['measurement_id'], inplace=True)
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species_name
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
#rename the species_name column just species 
iris_df.rename(columns={'species_name': 'species'}, inplace=True)
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [22]:
#Create dummy variables of the species name 
dummy_iris = pd.get_dummies(iris_df[['species']], dummy_na=False, drop_first=True)
dummy_iris.head()

,species_versicolor,species_virginica
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [24]:
#concatenate onto the iris dataframe. 
iris_df = pd.concat([iris_df, dummy_iris], axis=1)
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,species_versicolor,species_virginica,species_versicolor,species_virginica
0,5.1,3.5,1.4,0.2,setosa,0,0,0,0
1,4.9,3.0,1.4,0.2,setosa,0,0,0,0
2,4.7,3.2,1.3,0.2,setosa,0,0,0,0
3,4.6,3.1,1.5,0.2,setosa,0,0,0,0
4,5.0,3.6,1.4,0.2,setosa,0,0,0,0


In [25]:
#Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied. 
def prep_iris(iris_df):
    iris_df.drop(columns=['species_id'], inplace=True)
    iris_df.drop(columns=['measurement_id'], inplace=True)
    iris_df.rename(columns={'species_name':'species'}, inplace=True)
    dummy_iris = pd.get_dummies(iris_df[['species']], dummy_na=False, drop_first=[True])
    iris_df = pd.concat([iris_df, dummy_iris], axis=1)
    return iris_df
    

### Titanic Data Cleaning

In [12]:
titanic_df = acquire.get_titanic_data()

Acquiring data from SQL database


In [13]:
titanic_df.head(3)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1


In [26]:
#drop any unnecessary, unhelpful, or duplicated columns 
titanic_df.shape

(891, 13)

In [28]:
titanic_df.columns

Index(['passenger_id', 'survived', 'pclass', 'sex', 'age', 'sibsp', 'parch',
       'fare', 'embarked', 'class', 'deck', 'embark_town', 'alone'],
      dtype='object')

In [30]:
titanic_df.dtypes

passenger_id      int64
survived          int64
pclass            int64
sex              object
age             float64
sibsp             int64
parch             int64
fare            float64
embarked         object
class            object
deck             object
embark_town      object
alone             int64
dtype: object

In [31]:
titanic_df.describe(include='all')

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
count,891.000000,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,889,891,203,889,891.000000
unique,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,7,3,NaN
top,NaN,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,C,Southampton,NaN
freq,NaN,NaN,NaN,577,NaN,NaN,NaN,NaN,644,491,59,644,NaN
mean,445.000000,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN,NaN,NaN,NaN,0.602694
std,257.353842,0.486592,0.836071,NaN,14.526497,1.102743,0.806057,49.693429,NaN,NaN,NaN,NaN,0.489615
min,0.000000,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
25%,222.500000,0.000000,2.000000,NaN,20.125000,0.000000,0.000000,7.910400,NaN,NaN,NaN,NaN,0.000000
50%,445.000000,0.000000,3.000000,NaN,28.000000,0.000000,0.000000,14.454200,NaN,NaN,NaN,NaN,1.000000
75%,667.500000,1.000000,3.000000,NaN,38.000000,1.000000,0.000000,31.000000,NaN,NaN,NaN,NaN,1.000000


In [32]:
titanic_df.isnull().sum()

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
deck            688
embark_town       2
alone             0
dtype: int64

In [33]:
#drop the deck column too many nulls to be useful 
titanic_df.drop(columns=['deck'], inplace=True)
titanic_df.head(3)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,Southampton,1


In [34]:
titanic_df.embark_town.value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

In [37]:
#fill the 2 null in embark_town with 'Southhampton'
titanic_df['embark_town'] = titanic_df.embark_town.fillna(value="Southampton")

In [38]:
#fill with zeroes 
titanic_df['age'] = titanic_df.age.fillna(0)

In [39]:
titanic_df.isnull().sum()

passenger_id    0
survived        0
pclass          0
sex             0
age             0
sibsp           0
parch           0
fare            0
embarked        2
class           0
embark_town     0
alone           0
dtype: int64

In [44]:
#separate into train, validate, and test set 
train, test = train_test_split(titanic_df, train_size=0.7, random_state=1349, stratify=titanic_df.survived)

In [45]:
train.shape, test.shape

((623, 12), (268, 12))

In [14]:
telco_df = acquire.get_telco_data()

Acquiring data from SQL database


In [15]:
telco_df.head()

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,DSL,One year,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,DSL,Month-to-month,Mailed check
2,1,1,2,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Fiber optic,Month-to-month,Electronic check
3,1,1,2,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Fiber optic,Month-to-month,Electronic check
4,2,1,2,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Fiber optic,Month-to-month,Mailed check
